# Transfer Learning with TensorFlow Part 2: Fine-tuning

## Creating helper functions

So it's a good idea to put functions you'll want to use again in a script you can download and import into your notebooks (or elsewhere).

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-12-31 21:40:58--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2024-12-31 21:40:58 (7.01 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [1]:
# Import helper functions we're going to use in this notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

## Let's get some data

This time we're going to see how we can use the pretrained models within tf.keras.applications and apply them to our own problem (recognizing images of food).

In [2]:
# Get 10% of training data of 10 classes of Food101
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

unzip_data("10_food_classes_10_percent.zip")

--2025-01-01 13:15:42--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.76.123, 142.251.221.91, 142.250.71.91, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.76.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  12.4MB/s    in 14s     

2025-01-01 13:15:56 (11.8 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [2]:
# Check out how many images and subdirectories are in our dataset
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test

In [3]:
# Create training and test directories
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

In [4]:
import tensorflow as tf
tf.__version__

'2.16.2'

In [5]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                            image_size=IMG_SIZE,
                                                                            label_mode="categorical",
                                                                            batch_size=BATCH_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                image_size=IMG_SIZE,
                                                                label_mode="categorical",
                                                                batch_size=BATCH_SIZE)

Found 750 files belonging to 10 classes.


2025-01-01 13:50:17.883095: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-01-01 13:50:17.883125: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-01-01 13:50:17.883133: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-01-01 13:50:17.883170: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-01 13:50:17.883190: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 2500 files belonging to 10 classes.


In [6]:
# Check out the class names of our dataset
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [8]:
# See an example batch of data
for images, labels in train_data_10_percent.take(1):
    print(images, labels)

tf.Tensor(
[[[[3.00000000e+00 2.00000000e+00 0.00000000e+00]
   [3.00000000e+00 2.00000000e+00 0.00000000e+00]
   [3.00000000e+00 2.00000000e+00 0.00000000e+00]
   ...
   [0.00000000e+00 1.00000000e+01 4.20000000e+01]
   [1.61034703e-01 1.01610346e+01 4.36747627e+01]
   [3.57142806e-01 1.03571424e+01 4.62394905e+01]]

  [[2.07142830e+00 1.07142830e+00 0.00000000e+00]
   [2.07142830e+00 1.07142830e+00 0.00000000e+00]
   [2.07142830e+00 1.07142830e+00 0.00000000e+00]
   ...
   [0.00000000e+00 1.10000000e+01 4.10121193e+01]
   [3.22068222e-02 1.05813093e+01 4.16581688e+01]
   [7.14282990e-02 1.00714283e+01 4.22857132e+01]]

  [[2.00000000e+00 1.00000000e+00 0.00000000e+00]
   [2.00000000e+00 1.00000000e+00 0.00000000e+00]
   [2.00000000e+00 1.00000000e+00 0.00000000e+00]
   ...
   [0.00000000e+00 1.05714283e+01 3.63571434e+01]
   [0.00000000e+00 1.01527386e+01 3.67436256e+01]
   [0.00000000e+00 9.64285755e+00 3.72142868e+01]]

  ...

  [[1.03214264e+02 7.82142639e+01 2.42142639e+01]
   [1

2025-01-01 13:58:12.169583: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
